# Initializing Spark 

In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import findspark
findspark.init()
findspark.find()
import pyspark

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)
sqlcontext=SQLContext(sc)

# From Book Reference

## JOINS

In [2]:
# manually creating a dataframe

## Syntax
## DF=spark.createDataFrame(RDD,Columns)
## preferable

personDF=spark.createDataFrame([(0,"Bill chambers",0),(1,"Matt Armbrust",1)]).toDF("id","name","Graduate_program")

In [3]:
personDF.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- Graduate_program: long (nullable = true)



In [4]:
## Second way
personRDD=sc.parallelize([(0,"Bill chambers",0),(1,"Matt Armbrust",1)])
personCol=["id","name","Graduate_program"]

In [5]:
personDF=spark.createDataFrame(personRDD,personCol)

In [6]:
personDF.show()

+---+-------------+----------------+
| id|         name|Graduate_program|
+---+-------------+----------------+
|  0|Bill chambers|               0|
|  1|Matt Armbrust|               1|
+---+-------------+----------------+



In [7]:
personDF.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- Graduate_program: long (nullable = true)



In [8]:
#Manually Create two dataframes and join them
person = spark.createDataFrame([\
(0, "Bill Chambers", 0, [100]),\
(1, "Matei Zaharia", 1, [500, 250, 100]),\
(2, "Michael Armbrust", 1, [250, 100])])\
.toDF("id", "name", "graduate_program", "spark_status")

In [9]:
person.show()

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+



In [10]:
# Manually Creating Dataframes

graduateProgram = spark.createDataFrame([\
(0, "Masters", "School of Information", "UC Berkeley"),\
(2, "Masters", "EECS", "UC Berkeley"),\
(1, "Ph.D.", "EECS", "UC Berkeley")])\
.toDF("id", "degree", "department", "school")

In [11]:
graduateProgram.show()

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



In [12]:
sparkStatus = spark.createDataFrame([\
(500, "Vice President"),\
(250, "PMC Member"),\
(100, "Contributor")])\
.toDF("id", "status")

In [13]:
sparkStatus.show(),graduateProgram.show(),person.show()

+---+--------------+
| id|        status|
+---+--------------+
|500|Vice President|
|250|    PMC Member|
|100|   Contributor|
+---+--------------+

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+



(None, None, None)

In [14]:
#normal join example 
person.join(graduateProgram,person["graduate_program"]==graduateProgram["id"]).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



# From the lecture notes

#### Performing Joins with spark queries

In [15]:
#creating two dataframes to perform join operations
person_DF=spark.createDataFrame([(0,"Bill chambers",0),\
                                 (1,"Matt Armbrust",1)]).toDF("id","name","graduate_program")


In [16]:
program_DF=spark.createDataFrame([(0,"Masters","MIT"),\
                                  (1,"PHD","MIT"),\
                                  (2,"PHD","UCB")]).toDF("id","degree","school")


In [17]:
# the two dataframes are as follows
person_DF.show(),program_DF.show() #Left=person and right=program datasets

+---+-------------+----------------+
| id|         name|graduate_program|
+---+-------------+----------------+
|  0|Bill chambers|               0|
|  1|Matt Armbrust|               1|
+---+-------------+----------------+

+---+-------+------+
| id| degree|school|
+---+-------+------+
|  0|Masters|   MIT|
|  1|    PHD|   MIT|
|  2|    PHD|   UCB|
+---+-------+------+



(None, None)

In [18]:
#########For later
# creating tempView for performing join through SQL queries
#person_DF.createOrReplaceTempView("person")
#program_DF.createOrReplaceTempView("program")

### Natural Join: It joins the left and right datasets based on the common keys/column names

In [19]:
# performing "natural" join operation with spark query
person_DF.show(),program_DF.show()
person_DF.join(program_DF,person_DF["graduate_program"]==program_DF["id"])


+---+-------------+----------------+
| id|         name|graduate_program|
+---+-------------+----------------+
|  0|Bill chambers|               0|
|  1|Matt Armbrust|               1|
+---+-------------+----------------+

+---+-------+------+
| id| degree|school|
+---+-------+------+
|  0|Masters|   MIT|
|  1|    PHD|   MIT|
|  2|    PHD|   UCB|
+---+-------+------+



DataFrame[id: bigint, name: string, graduate_program: bigint, id: bigint, degree: string, school: string]

### Inner Joins: Based on the keys in the left and datasets, joins the rows with common keys and discards the rest

In [20]:
# performing "inner" join operation with spark query 
person_DF.show(),program_DF.show()
person_DF.join(program_DF,person_DF["graduate_program"]==program_DF["id"],"inner").show()



+---+-------------+----------------+
| id|         name|graduate_program|
+---+-------------+----------------+
|  0|Bill chambers|               0|
|  1|Matt Armbrust|               1|
+---+-------------+----------------+

+---+-------+------+
| id| degree|school|
+---+-------+------+
|  0|Masters|   MIT|
|  1|    PHD|   MIT|
|  2|    PHD|   UCB|
+---+-------+------+

+---+-------------+----------------+---+-------+------+
| id|         name|graduate_program| id| degree|school|
+---+-------------+----------------+---+-------+------+
|  0|Bill chambers|               0|  0|Masters|   MIT|
|  1|Matt Armbrust|               1|  1|    PHD|   MIT|
+---+-------------+----------------+---+-------+------+



### Outer Joins: keeps all the keys in the left and right datasets

In [21]:
# performing "outer" join operation with spark query 
person_DF.show(),program_DF.show()
person_DF.join(program_DF,person_DF["graduate_program"]==program_DF["id"],"outer").show()


+---+-------------+----------------+
| id|         name|graduate_program|
+---+-------------+----------------+
|  0|Bill chambers|               0|
|  1|Matt Armbrust|               1|
+---+-------------+----------------+

+---+-------+------+
| id| degree|school|
+---+-------+------+
|  0|Masters|   MIT|
|  1|    PHD|   MIT|
|  2|    PHD|   UCB|
+---+-------+------+

+----+-------------+----------------+---+-------+------+
|  id|         name|graduate_program| id| degree|school|
+----+-------------+----------------+---+-------+------+
|   0|Bill chambers|               0|  0|Masters|   MIT|
|   1|Matt Armbrust|               1|  1|    PHD|   MIT|
|null|         null|            null|  2|    PHD|   UCB|
+----+-------------+----------------+---+-------+------+



### Left Outer Join: Matches the keys in the left and right datasets and only keeps the rows from the left dataset. keeps the rows with keys in the left dataset

In [22]:
# left outer join
program_DF.show(),person_DF.show()
program_DF.join(person_DF,person_DF["graduate_program"]==program_DF["id"],"left_outer").show()



+---+-------+------+
| id| degree|school|
+---+-------+------+
|  0|Masters|   MIT|
|  1|    PHD|   MIT|
|  2|    PHD|   UCB|
+---+-------+------+

+---+-------------+----------------+
| id|         name|graduate_program|
+---+-------------+----------------+
|  0|Bill chambers|               0|
|  1|Matt Armbrust|               1|
+---+-------------+----------------+

+---+-------+------+----+-------------+----------------+
| id| degree|school|  id|         name|graduate_program|
+---+-------+------+----+-------------+----------------+
|  0|Masters|   MIT|   0|Bill chambers|               0|
|  1|    PHD|   MIT|   1|Matt Armbrust|               1|
|  2|    PHD|   UCB|null|         null|            null|
+---+-------+------+----+-------------+----------------+



### Right outer join: Keeps the rows with keys in the right dataset 

In [23]:
# right outer join
program_DF.show(),person_DF.show()
program_DF.join(person_DF,person_DF["graduate_program"]==program_DF["id"],"right_outer").show()


+---+-------+------+
| id| degree|school|
+---+-------+------+
|  0|Masters|   MIT|
|  1|    PHD|   MIT|
|  2|    PHD|   UCB|
+---+-------+------+

+---+-------------+----------------+
| id|         name|graduate_program|
+---+-------------+----------------+
|  0|Bill chambers|               0|
|  1|Matt Armbrust|               1|
+---+-------------+----------------+

+---+-------+------+---+-------------+----------------+
| id| degree|school| id|         name|graduate_program|
+---+-------+------+---+-------------+----------------+
|  0|Masters|   MIT|  0|Bill chambers|               0|
|  1|    PHD|   MIT|  1|Matt Armbrust|               1|
+---+-------+------+---+-------------+----------------+



### Left semi join: matches the keys and only keeps the rows with common keys in the left dataset

In [24]:
# performing "left semi" join operation with spark query 
program_DF.show(),person_DF.show()
program_DF.join(person_DF,person_DF["graduate_program"]==program_DF["id"],"left_semi").show()



+---+-------+------+
| id| degree|school|
+---+-------+------+
|  0|Masters|   MIT|
|  1|    PHD|   MIT|
|  2|    PHD|   UCB|
+---+-------+------+

+---+-------------+----------------+
| id|         name|graduate_program|
+---+-------------+----------------+
|  0|Bill chambers|               0|
|  1|Matt Armbrust|               1|
+---+-------------+----------------+

+---+-------+------+
| id| degree|school|
+---+-------+------+
|  0|Masters|   MIT|
|  1|    PHD|   MIT|
+---+-------+------+



### Left anti join: works opposite to left semi join. compares the keys and keeps only those rows which donot match in the left dataset 

In [25]:
# performing "left anti" join operation with spark query 
program_DF.show(),person_DF.show()
program_DF.join(person_DF,person_DF["graduate_program"]==program_DF["id"],"left_anti").show()


+---+-------+------+
| id| degree|school|
+---+-------+------+
|  0|Masters|   MIT|
|  1|    PHD|   MIT|
|  2|    PHD|   UCB|
+---+-------+------+

+---+-------------+----------------+
| id|         name|graduate_program|
+---+-------------+----------------+
|  0|Bill chambers|               0|
|  1|Matt Armbrust|               1|
+---+-------------+----------------+

+---+------+------+
| id|degree|school|
+---+------+------+
|  2|   PHD|   UCB|
+---+------+------+



In [26]:
# Cross join with Spark query
c1_df=spark.createDataFrame([("a"),\
                             ("b")],"string").toDF("C1")

In [27]:
c2_df=spark.createDataFrame([("c"),\
                             ("d"),\
                             ("e")],"string").toDF("C2")

In [28]:
c1_df.show(),c2_df.show()

+---+
| C1|
+---+
|  a|
|  b|
+---+

+---+
| C2|
+---+
|  c|
|  d|
|  e|
+---+



(None, None)

###  Cross join or cartesian: it is the cartesian product of both the datasets

In [29]:
# Applying Cross join
c1_df.show(),c2_df.show()
c2_df.crossJoin(c1_df).show()

+---+
| C1|
+---+
|  a|
|  b|
+---+

+---+
| C2|
+---+
|  c|
|  d|
|  e|
+---+

+---+---+
| C2| C1|
+---+---+
|  c|  a|
|  c|  b|
|  d|  a|
|  d|  b|
|  e|  a|
|  e|  b|
+---+---+



#### Performing joins with SQL queries


In [30]:
# creating tempView for performing join through SQL queries
person_DF.createOrReplaceTempView("person")
program_DF.createOrReplaceTempView("program")

In [31]:
# Natural Join
# Left dataset= Person, Right dataset= program
spark.sql("""SELECT * FROM person NATURAL JOIN program WHERE (person.graduate_program)==(program.id) ;""").explain()

== Physical Plan ==
*(5) Project [id#203L, name#204, graduate_program#205L, degree#216, school#217]
+- *(5) SortMergeJoin [graduate_program#205L, id#203L], [id#215L, id#215L], Inner
   :- *(2) Sort [graduate_program#205L ASC NULLS FIRST, id#203L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(graduate_program#205L, id#203L, 200), ENSURE_REQUIREMENTS, [id=#756]
   :     +- *(1) Project [_1#197L AS id#203L, _2#198 AS name#204, _3#199L AS graduate_program#205L]
   :        +- *(1) Filter (((_1#197L = _3#199L) AND isnotnull(_3#199L)) AND isnotnull(_1#197L))
   :           +- *(1) Scan ExistingRDD[_1#197L,_2#198,_3#199L]
   +- *(4) Sort [id#215L ASC NULLS FIRST, id#215L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(id#215L, id#215L, 200), ENSURE_REQUIREMENTS, [id=#762]
         +- *(3) Project [_1#209L AS id#215L, _2#210 AS degree#216, _3#211 AS school#217]
            +- *(3) Filter isnotnull(_1#209L)
               +- *(3) Scan ExistingRDD[_1#209L,_2#210,_3#2

In [32]:
# Inner Joins
spark.sql("""SELECT * FROM person INNER JOIN program ON (person.graduate_program)==(program.id) ;""").show()

+---+-------------+----------------+---+-------+------+
| id|         name|graduate_program| id| degree|school|
+---+-------------+----------------+---+-------+------+
|  0|Bill chambers|               0|  0|Masters|   MIT|
|  1|Matt Armbrust|               1|  1|    PHD|   MIT|
+---+-------------+----------------+---+-------+------+



In [33]:
# Outer Join
spark.sql("""SELECT * FROM person FULL OUTER JOIN program ON program.id=person.graduate_program ;""").show()

+----+-------------+----------------+---+-------+------+
|  id|         name|graduate_program| id| degree|school|
+----+-------------+----------------+---+-------+------+
|   0|Bill chambers|               0|  0|Masters|   MIT|
|   1|Matt Armbrust|               1|  1|    PHD|   MIT|
|null|         null|            null|  2|    PHD|   UCB|
+----+-------------+----------------+---+-------+------+



In [34]:
# Left Outer Join
spark.sql("""SELECT * FROM program LEFT OUTER JOIN person ON (program.id)=(person.graduate_program) ;""").show()


+---+-------+------+----+-------------+----------------+
| id| degree|school|  id|         name|graduate_program|
+---+-------+------+----+-------------+----------------+
|  0|Masters|   MIT|   0|Bill chambers|               0|
|  1|    PHD|   MIT|   1|Matt Armbrust|               1|
|  2|    PHD|   UCB|null|         null|            null|
+---+-------+------+----+-------------+----------------+



In [35]:
# right outer
spark.sql("""SELECT * FROM program RIGHT OUTER JOIN person ON (program.id)=(person.graduate_program) ;""").show()


+---+-------+------+---+-------------+----------------+
| id| degree|school| id|         name|graduate_program|
+---+-------+------+---+-------------+----------------+
|  0|Masters|   MIT|  0|Bill chambers|               0|
|  1|    PHD|   MIT|  1|Matt Armbrust|               1|
+---+-------+------+---+-------------+----------------+



In [36]:
# left semi
spark.sql("""SELECT * FROM program LEFT SEMI JOIN person ON (program.id)=(person.graduate_program) ;""").show()


+---+-------+------+
| id| degree|school|
+---+-------+------+
|  0|Masters|   MIT|
|  1|    PHD|   MIT|
+---+-------+------+



In [37]:
# Left anti
spark.sql("""SELECT * FROM program LEFT ANTI JOIN person ON (program.id)=(person.graduate_program) ;""").show()


+---+------+------+
| id|degree|school|
+---+------+------+
|  2|   PHD|   UCB|
+---+------+------+



# Programming Questions

###  Pivot

#### For this section, you are provided with the following retails dataset. Please use the dataset to answer the following question.

Download the dataset from here: https://drive.google.com/file/d/13y81xA5ilsse4jE9HWtLkwf-3-_Lw3-1/view?usp=sharing

If the link does not work, the dataset is available in the materials folder.

 

Q: How many instances of each product were sold in each country?



In [38]:

retails_data=spark.read.csv("2010-12-01.json",header=True,inferSchema=True)

In [39]:
retails_data.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [40]:
retails_data.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows



In [41]:
retails_data.groupBy("StockCode").pivot("Country").agg({"Quantity":"sum"}).orderBy("StockCode").show()

+---------+---------+----+------+-------+-----------+------+--------------+
|StockCode|Australia|EIRE|France|Germany|Netherlands|Norway|United Kingdom|
+---------+---------+----+------+-------+-----------+------+--------------+
|    10002|     null|null|    48|   null|       null|  null|            12|
|    10125|     null|null|  null|   null|       null|  null|             2|
|    10133|     null|null|  null|   null|       null|  null|             5|
|    10135|     null|null|  null|   null|       null|  null|             1|
|    11001|     null|null|  null|   null|       null|  null|             3|
|   15044B|     null|null|  null|   null|       null|  null|             1|
|  15056BL|     null|null|  null|   null|       null|  null|            20|
|   15056N|     null|null|  null|   null|       null|  null|            50|
|   15056P|     null|null|  null|   null|       null|  null|            48|
|    16014|     null|null|  null|   null|       null|  null|            10|
|    16016| 

### Joins - Refer to the schema in the assignment

The following schema describes customers who can place orders. Furthermore, there are employees who can convince customers to place an order. Not every order has to have a corresponding salesman. In this case, NULL is stored in the salesman id column. Employees can have supervisors. The orders table is the fact table and the other tables are dimensional tables. Use this schema for the following tasks. The DataFrames Customers, Orders, and Employees are available as variable and view with the names customers, orders, and employees respectively.

    Q: Which employees could convince customers to order products?


 

 

 

 

 

In [42]:
employees=spark.createDataFrame([(0,"John","Brick","DonJohn@abc.com","01234","23-01-2000",20,"HR"),\
                                (1,"Doe","Moe","DoeJoe@xyz.com","345123","03-07-2010",20,"Sales")]).\
                                    toDF("Employee_ID","First_name","Last_name","email","phone",\
                                         "hire_date","Manager_ID","job_title")


In [43]:
orders=spark.createDataFrame([(22,552,"Availaible",0,"31-12-2021"),\
                              (28,672,"Availaible",1,"29-02-2020"),\
                              (60,560,"Not Availaible",2,"01-01-2021")])\
                             .toDF("Order_ID","customer_ID","status","salesman_ID","order_date")

In [44]:
customers=spark.createDataFrame([(560,"rico","new york","na","no limit")\
                                ,(600,"kayle","Hamburg","na","exceeded")\
                                ,(552,"pepe","barcelona","na","available")]).toDF("customer_ID","name","address","website","credit_limit")

In [45]:
employees.show(),orders.show(),customers.show()

+-----------+----------+---------+---------------+------+----------+----------+---------+
|Employee_ID|First_name|Last_name|          email| phone| hire_date|Manager_ID|job_title|
+-----------+----------+---------+---------------+------+----------+----------+---------+
|          0|      John|    Brick|DonJohn@abc.com| 01234|23-01-2000|        20|       HR|
|          1|       Doe|      Moe| DoeJoe@xyz.com|345123|03-07-2010|        20|    Sales|
+-----------+----------+---------+---------------+------+----------+----------+---------+

+--------+-----------+--------------+-----------+----------+
|Order_ID|customer_ID|        status|salesman_ID|order_date|
+--------+-----------+--------------+-----------+----------+
|      22|        552|    Availaible|          0|31-12-2021|
|      28|        672|    Availaible|          1|29-02-2020|
|      60|        560|Not Availaible|          2|01-01-2021|
+--------+-----------+--------------+-----------+----------+

+-----------+-----+---------+--

(None, None, None)

In [46]:
join_expression=customers["customer_ID"]==orders["customer_ID"]

In [47]:
customer_orders=orders.join(customers,join_expression)

In [48]:
employees.show(),customer_orders.show()

+-----------+----------+---------+---------------+------+----------+----------+---------+
|Employee_ID|First_name|Last_name|          email| phone| hire_date|Manager_ID|job_title|
+-----------+----------+---------+---------------+------+----------+----------+---------+
|          0|      John|    Brick|DonJohn@abc.com| 01234|23-01-2000|        20|       HR|
|          1|       Doe|      Moe| DoeJoe@xyz.com|345123|03-07-2010|        20|    Sales|
+-----------+----------+---------+---------------+------+----------+----------+---------+

+--------+-----------+--------------+-----------+----------+-----------+----+---------+-------+------------+
|Order_ID|customer_ID|        status|salesman_ID|order_date|customer_ID|name|  address|website|credit_limit|
+--------+-----------+--------------+-----------+----------+-----------+----+---------+-------+------------+
|      22|        552|    Availaible|          0|31-12-2021|        552|pepe|barcelona|     na|   available|
|      60|        560|N

(None, None)

In [49]:
join_expression=employees["employee_ID"]==customer_orders["salesman_ID"]

In [50]:
employees.join(customer_orders,join_expression,"leftsemi").show()

+-----------+----------+---------+---------------+-----+----------+----------+---------+
|Employee_ID|First_name|Last_name|          email|phone| hire_date|Manager_ID|job_title|
+-----------+----------+---------+---------------+-----+----------+----------+---------+
|          0|      John|    Brick|DonJohn@abc.com|01234|23-01-2000|        20|       HR|
+-----------+----------+---------+---------------+-----+----------+----------+---------+



In [51]:
#using sql query to join
employees.createOrReplaceTempView("employees")
customers.createOrReplaceTempView("customers")
orders.createOrReplaceTempView("orders")

In [52]:
customer_orders=spark.sql("""SELECT * FROM orders INNER JOIN customers\
                            ON (orders.customer_ID)=(customers.customer_ID) ;""")

In [53]:
customer_orders.createOrReplaceTempView("customer_orders")

In [55]:
spark.sql("""SELECT * FROM employees LEFT SEMI JOIN customer_orders \
        ON (employees.Employee_ID)=(customer_orders.salesman_ID) ;""").explain()

== Physical Plan ==
*(9) Project [_1#12198L AS Employee_ID#12214L, _2#12199 AS First_name#12215, _3#12200 AS Last_name#12216, _4#12201 AS email#12217, _5#12202 AS phone#12218, _6#12203 AS hire_date#12219, _7#12204L AS Manager_ID#12220L, _8#12205 AS job_title#12221]
+- SortMergeJoin [_1#12198L], [salesman_ID#12243L], LeftSemi
   :- *(2) Sort [_1#12198L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(_1#12198L, 200), ENSURE_REQUIREMENTS, [id=#1580]
   :     +- *(1) Filter isnotnull(_1#12198L)
   :        +- *(1) Scan ExistingRDD[_1#12198L,_2#12199,_3#12200,_4#12201,_5#12202,_6#12203,_7#12204L,_8#12205]
   +- *(8) Sort [salesman_ID#12243L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(salesman_ID#12243L, 200), ENSURE_REQUIREMENTS, [id=#1599]
         +- *(7) Project [salesman_ID#12243L]
            +- *(7) SortMergeJoin [customer_ID#12241L], [customer_ID#12260L], Inner
               :- *(4) Sort [customer_ID#12241L ASC NULLS FIRST], false, 0
               : 